In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

#from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling3D , Dropout, Conv2D,MaxPool2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import os, glob, random, math
import shutil

from shutil import copyfile,copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
IMG_SHAPE = (150,150,1)
IMG_TUPPLE_SHAPE = (150,150)
BATCH_SIZE= 128

In [4]:
! git clone https://github.com/NACIONES-DESUNIDAS/Dataset_Raw.git

Cloning into 'Dataset_Raw'...
remote: Enumerating objects: 3703, done.
remote: Counting objects: 100% (3703/3703), done.
remote: Compressing objects: 100% (3695/3695), done.
remote: Total 3703 (delta 8), reused 3703 (delta 8), pack-reused 0
Receiving objects: 100% (3703/3703), 26.97 MiB | 29.38 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [5]:
DATASET_NAME = "Dataset_Raw"
DRIVE_PATH = "/content/drive/Shareddrives/Semestre6"
DATASET_PATH = os.path.join("/content/",DATASET_NAME)
MODEL_CHECKPOINTS = "MODEL_CHECKPOINTS_OCTAVIO"
MODEL_SAVE_DIRECTORY = os.path.join(DRIVE_PATH,MODEL_CHECKPOINTS)
DATASET_FOLDERS = ["00014","00032","00033","00034","00035","00040"]
IMAGE_FOLDER_PATHS = [os.path.join(os.path.join(DATASET_PATH,folder),"*.*") for folder in DATASET_FOLDERS]
IMAGE_LABELS = ["stop_signal","aplastame","right_signal","left_signal","up_signal","around_signal"]  ###
RESET = "/content/"

In [6]:
IMAGE_FOLDER_PATHS

['/content/Dataset_Raw/00014/*.*',
 '/content/Dataset_Raw/00032/*.*',
 '/content/Dataset_Raw/00033/*.*',
 '/content/Dataset_Raw/00034/*.*',
 '/content/Dataset_Raw/00035/*.*',
 '/content/Dataset_Raw/00040/*.*']

In [7]:
imagePathLists = [glob.glob(path) for path in IMAGE_FOLDER_PATHS]

In [8]:
def imageProcessing(image):
  image = image.astype(np.uint8)
  #gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  gray = cv2.equalizeHist(image)
  gray = cv2.resize(gray,(150,150))
  gray = gray.reshape(150,150,1)
  return gray.astype(np.float64)/255

In [9]:
def readImages(imageList):
  images = list()
  for path in imageList:
    if path.split(".")[-1] == "csv":
      continue
    image = cv2.imread(path)
    image = imageProcessing(image)
    images.append(image)
  return images

In [10]:
def imageDataAumentationReshape(img):
    return img.reshape(150,150,1)

In [11]:
def getImageDataframe(imageList,index):
  labels = [IMAGE_LABELS[index]] * len(imageList)
  return pd.DataFrame({"images":imageList,"label":labels})
  
  


In [12]:
# obtener una lista de listas de imagenes preprocesadas por cada folder
#imageLists = [readImages(pathList) for pathList in imagePathLists]

In [13]:
#len(imageLists)

In [14]:
# obtener una lista de dataframes donde se agrupa cada imagen con su label respectivo
dataFrames = [getImageDataframe(imagePathLists[i],i) for i in range(len(imagePathLists))]

In [15]:
def mergeDataFrames(dataFrames):
  initial = dataFrames[0]
  for i in range(1,len(dataFrames)):
    initial = pd.merge(initial, dataFrames[i],  how='outer', left_on=['labels'], right_on = ['labels'])


In [16]:
dataset = pd.concat(dataFrames,ignore_index=True)

In [17]:
dataset.head()

,images,label
0,/content/Dataset_Raw/00014/00000_00023.ppm,stop_signal
1,/content/Dataset_Raw/00014/00004_00002.ppm,stop_signal
2,/content/Dataset_Raw/00014/00016_00021.ppm,stop_signal
3,/content/Dataset_Raw/00014/00001_00015.ppm,stop_signal
4,/content/Dataset_Raw/00014/00016_00016.ppm,stop_signal


In [18]:
class2Label = dict()
label2Class = dict()
for i in range(len(IMAGE_LABELS)):
  class2Label[IMAGE_LABELS[i]] = i+1
  label2Class[i+1] = IMAGE_LABELS[i]

In [19]:
print(class2Label)
print(label2Class)

{'stop_signal': 1, 'aplastame': 2, 'right_signal': 3, 'left_signal': 4, 'up_signal': 5, 'around_signal': 6}
{1: 'stop_signal', 2: 'aplastame', 3: 'right_signal', 4: 'left_signal', 5: 'up_signal', 6: 'around_signal'}


In [20]:
dataset["encoding"] = dataset["label"].apply(lambda x:class2Label[x])

In [21]:
dataset.head()

,images,label,encoding
0,/content/Dataset_Raw/00014/00000_00023.ppm,stop_signal,1
1,/content/Dataset_Raw/00014/00004_00002.ppm,stop_signal,1
2,/content/Dataset_Raw/00014/00016_00021.ppm,stop_signal,1
3,/content/Dataset_Raw/00014/00001_00015.ppm,stop_signal,1
4,/content/Dataset_Raw/00014/00016_00016.ppm,stop_signal,1


In [22]:
augmentedDataGen = ImageDataGenerator(
rotation_range = 40,
width_shift_range = 0.2,
height_shift_range = 0.2,
shear_range = 0.2,
zoom_range=0.2,
horizontal_flip=False,
vertical_flip = False,
validation_split=0.2,
fill_mode='nearest',
preprocessing_function = imageProcessing 

)


testDataGen = ImageDataGenerator(
preprocessing_function = imageProcessing 

)

In [23]:
dataset.columns

Index(['images', 'label', 'encoding'], dtype='object')

In [24]:
trainGenerator = augmentedDataGen.flow_from_dataframe(
    dataset,
    directory=None,
    x_col='images',
    y_col='label',
    target_size=(150, 150),
    classes=IMAGE_LABELS,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset = "training",
    color_mode = "grayscale"
)

validationGenerator = augmentedDataGen.flow_from_dataframe(
    dataset,
    directory=None,
    x_col='images',
    y_col='label',
    target_size=(150, 150),
    classes=IMAGE_LABELS,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset = "validation",
    color_mode = "grayscale"
    
)

testGenerator = testDataGen.flow_from_dataframe(
    dataset,
    directory=None,
    x_col='images',
    y_col='label',
    target_size=(150, 150),
    classes=IMAGE_LABELS,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset = "validation",
    color_mode = "grayscale"
    
)



Found 2952 validated image filenames belonging to 6 classes.
Found 737 validated image filenames belonging to 6 classes.
Found 0 validated image filenames belonging to 6 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 6 invalid image filename(s) in x_col="images". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [25]:

dropoutRate = 0.2
model = Sequential()

model.add(Conv2D(32,kernel_size=3,padding="SAME",activation="relu",input_shape = (150,150,1)))
model.add(MaxPool2D(pool_size = 2))
model.add(Conv2D(64,kernel_size=3,padding="SAME",activation="relu"))
model.add(MaxPool2D(pool_size = 2))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(dropoutRate))
model.add(Dense(32,activation="relu"))
model.add(Dense(6,activation="softmax"))

In [26]:
model.compile(
    optimizer = "rmsprop",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

In [27]:
def checkModelCheckpointsDir():
  if not os.path.exists(MODEL_SAVE_DIRECTORY):
    os.chdir(DRIVE_PATH)
    os.mkdir(MODEL_SAVE_DIRECTORY)
    os.chdir("/content/")

In [28]:
checkModelCheckpointsDir()

In [29]:
check = ModelCheckpoint(filepath=MODEL_SAVE_DIRECTORY,save_weights_only=False,frequency="epoch",monitor = "val_accuracy",save_best_only=True,verbose = 1)
stop = EarlyStopping(mode="max",patience = 10,min_delta = 0.05,monitor ="val_accuracy")

In [30]:
hist = model.fit(
  trainGenerator,
  epochs=100,
  validation_data=validationGenerator,
  steps_per_epoch=trainGenerator.samples // BATCH_SIZE,
  validation_steps=validationGenerator.samples // BATCH_SIZE,
  callbacks=[check,stop],
  verbose = 1
  )

Epoch 1/100
23/23 [==============================] - ETA: 0s - loss: 2.3995 - accuracy: 0.3541
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to /content/drive/Shareddrives/Semestre6/MODEL_CHECKPOINTS_OCTAVIO
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/Semestre6/MODEL_CHECKPOINTS_OCTAVIO/assets
23/23 [==============================] - 23s 400ms/step - loss: 2.3995 - accuracy: 0.3541 - val_loss: 3.6549 - val_accuracy: 0.0000e+00
Epoch 2/100
23/23 [==============================] - ETA: 0s - loss: 1.5104 - accuracy: 0.3906
Epoch 2: val_accuracy did not improve from 0.00000
23/23 [==============================] - 8s 329ms/step - loss: 1.5104 - accuracy: 0.3906 - val_loss: 4.2407 - val_accuracy: 0.0000e+00
Epoch 3/100
23/23 [==============================] - ETA: 0s - loss: 1.4760 - accuracy: 0.4044
Epoch 3: val_accuracy did not improve from 0.00000
23/23 [==============================] - 8s 349ms/step - loss: 1.4760 - accuracy: 0.4044 - val_loss: 3.

KeyboardInterrupt: ignored